In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# Constants
TOTAL_FIELDS = 259
DATADIR = '../data/'
RESULTDIR = '../results/'
FILE = 'uniswap.csv'

In [4]:
# Functions
def get_gas_table():
    gas_price_data = read_csv(DATADIR + 'gas_table.csv')
    gas_table = {}
    for fields in gas_price_data:
        if len(fields) == 4:
            gas_table[int(fields[0], 16)] = {
                'current_gas_cost': fields[1],
                'proposed_gas_cost': fields[2],
                'mnemonic': fields[3],
            }
    return gas_table

# Calculates new gas used based on the gas_table
def apply_gas_table(line):
    gas_table = get_gas_table()
    for ix, op in enumerate(line):
        print(ix, op)

In [5]:
def read_csv(file):
    fin = open(file, 'r')
    data = fin.read()
    fin.close()
    data = data.split('\n')
    data = [line.split(',') for line  in data]
    return data

In [6]:
def convert_data(data):
    final_data = []
    for line in data:
        new_line = [int(elem) if i > 1 else elem for i, elem in enumerate(line)]
        final_data.append(new_line)
    return final_data
        
def apply_gas_table(gas_table, line):
    if len(line) != TOTAL_FIELDS:
        return None
    
    tx_gas_used = int(line[2])
    
    if tx_gas_used == 0:
        print('Ignoring transaction', line[1], 'gasUsed:', tx_gas_used)
        return None
    
    opdata = line[3:]
    
    tx_total = 0
    tx_new_total = 0
    line_out = []
    for ix, count in enumerate(opdata):
        opnum = ix
        opname = columns_name[opnum+3]
        if ix in gas_table.keys():
            cost = gas_table[opnum]['current_gas_cost']
            total = int(count) * int(cost)
            new_cost = gas_table[opnum]['proposed_gas_cost']
            new_total = int(count) * int(new_cost)
            
            tx_total += total
            tx_new_total += new_total
            
    diff = tx_total - tx_new_total
    # Substracts the difference to the tx gasUsage
    new_tx_gas_used = tx_gas_used - diff
    decrease = tx_gas_used - new_tx_gas_used
    decrease_perc = (decrease / float(tx_gas_used)) * 100
    
    #         [Block #, Tx, Gas Used] + [new_gas_used, difference, percentage] + [opdata]
    line_out = line[:3] + [new_tx_gas_used, diff, "{:.2f}%".format(decrease_perc)] + line[3:]
    return line_out


In [7]:
def write_csv(fname, data):
    fout = open(fname, 'w')
    fout.write(data)
    fout.close()    


def convert_to_csv(data):
    out_str = ''
    for line in data:
        line = [x if isinstance(x, str) else str(x) for x in line]
        out_str += ','.join(line) + '\n'
    return out_str

In [8]:
csv = read_csv(DATADIR + FILE)

In [9]:
columns_name = csv[0]
columns_value = csv[1]
data = convert_data(csv[2:])

In [10]:
# Show DataFrame with only opcodes with > 0 executions (Only a sample of 100 txs)
df = pd.DataFrame(data[:100], columns=columns_name)
df.loc[:, (df != 0).any(axis=0)]

,Block #,Transactions,Gas Used,STOP,ADD,MUL,SUB,DIV,EXP,LT,GT,SLT,EQ,ISZERO,AND,OR,NOT,BYTE,SHL,SHR,KECCAK256,ADDRESS,CALLER,CALLVALUE,CALLDATALOAD,CALLDATASIZE,CALLDATACOPY,CODECOPY,EXTCODESIZE,RETURNDATASIZE,RETURNDATACOPY,TIMESTAMP,POP,MLOAD,MSTORE,SLOAD,SSTORE,JUMP,JUMPI,GAS,JUMPDEST,PUSH1,PUSH2,PUSH3,PUSH4,PUSH5,PUSH6,PUSH8,PUSH9,PUSH10,PUSH11,PUSH12,PUSH13,PUSH14,PUSH15,PUSH18,PUSH20,PUSH21,PUSH28,PUSH29,PUSH31,PUSH32,DUP1,DUP2,DUP3,DUP4,DUP5,DUP6,DUP7,DUP8,DUP9,DUP10,DUP11,DUP12,DUP13,DUP14,DUP15,DUP16,SWAP1,SWAP2,SWAP3,SWAP4,SWAP5,SWAP6,SWAP7,SWAP8,SWAP9,SWAP10,SWAP11,SWAP12,SWAP13,SWAP14,SWAP15,LOG1,LOG2,LOG3,LOG4,CALL,RETURN,DELEGATECALL,STATICCALL,REVERT
0,0xbbcd65,0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90...,106777,2,187,39,47,22,1,56,36,2,61,117,177,9,3,0,13,5,18,2,6,9,27,13,3,3,6,8,1,2,351,114,126,21,10,122,218,7,288,414,348,1,85,4,0,1,0,0,0,8,1,19,5,0,76,0,8,7,0,30,239,212,150,102,88,41,17,15,18,9,6,3,1,2,0,0,231,139,85,27,18,14,6,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
1,0xbbcd65,0xef6102ed1c3a4841aaae5d6fbb0e5cf139584a0776be...,83381,1,210,38,47,26,5,54,32,0,75,127,175,9,5,0,11,3,18,2,7,7,28,10,3,0,4,11,2,2,352,115,128,20,9,110,224,6,267,424,343,1,94,4,0,1,0,0,0,0,0,19,5,0,80,0,9,8,0,31,254,208,157,95,83,37,22,12,17,9,7,5,1,2,0,0,236,156,82,19,14,13,6,4,8,2,3,0,0,0,0,1,0,3,0,3,6,0,3,0
2,0xbbcd65,0x8a7eec8c1e392d0f2d8178731061024bff3f64f4c0c9...,121450,2,195,39,45,25,2,55,31,0,82,127,172,7,3,0,13,3,17,2,9,9,27,11,3,0,6,8,1,2,345,111,120,19,10,104,229,7,261,415,339,1,106,4,0,1,0,0,0,0,0,19,5,0,83,0,8,9,0,28,250,208,149,89,81,35,17,13,18,9,6,3,1,2,0,0,218,135,77,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
3,0xbbcd65,0xc5ad659bbb28dc475f7cae798ae752c5cc07f5608aaf...,65866,2,158,28,47,17,20,33,15,0,86,93,110,3,4,1,7,2,16,1,8,7,44,12,6,1,5,5,1,1,280,100,123,20,0,82,187,8,220,419,280,0,97,2,1,1,0,0,0,0,1,3,1,0,45,0,1,7,1,20,203,184,81,86,56,25,26,24,18,8,4,2,0,0,0,0,189,109,47,14,19,17,11,4,5,1,2,0,0,0,0,1,0,0,0,6,6,1,1,1
4,0xbbcd65,0x2c015811f7241d46e583b449850af1d9faed32c036a1...,118690,2,183,39,52,22,1,57,33,0,61,115,154,7,3,0,18,5,17,2,9,9,27,13,3,0,6,8,1,2,334,112,122,18,10,104,211,7,261,412,320,1,84,4,0,1,0,0,0,0,0,19,5,0,65,0,8,7,0,29,228,214,146,88,82,36,16,13,18,9,6,3,1,2,0,0,212,124,81,21,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
5,0xbbcd65,0x20a4704294faecc2f3569764797fdc4a511cd8abef0d...,118762,2,195,39,47,22,1,57,32,0,64,118,169,7,3,0,13,5,16,2,6,9,27,13,3,0,6,8,1,2,351,111,118,17,10,104,213,7,262,408,322,1,87,4,0,1,0,0,0,0,0,19,5,0,83,0,8,7,0,28,242,212,147,88,84,36,17,13,18,9,6,3,1,2,0,0,213,135,79,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
6,0xbbcd65,0x32cd15e283acbb68c1f423b661feb49783c1c8914f23...,119028,2,195,39,47,24,1,57,30,0,76,126,171,7,3,0,13,3,16,2,6,9,27,13,3,0,6,8,1,2,351,111,118,17,10,104,223,7,262,406,332,1,99,4,0,1,0,0,0,0,0,19,5,0,83,0,8,9,0,28,252,212,147,88,84,36,17,13,18,9,6,3,1,2,0,0,215,135,79,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
7,0xbbcd65,0x911c9ece1b6418b58b9ae99a8ffef09dbf6ca018c01e...,145742,4,231,30,59,44,12,60,53,0,82,159,234,10,5,0,9,6,28,4,11,10,35,17,1,1,7,14,2,2,665,122,189,44,13,195,282,10,436,690,495,1,117,4,0,0,0,0,6,0,0,19,5,0,118,0,9,8,0,46,360,277,190,168,86,56,24,11,16,10,16,6,5,1,0,0,388,230,118,44,19,18,4,4,4,1,8,1,0,0,0,1,1,4,0,6,7,0,5,0
8,0xbbcd65,0xbd7c781dff21d7d5fa20734a3400fd58f05de9d8aad4...,159802,3,331,61,87,52,15,94,56,0,138,217,294,15,7,0,22,5,30,5,9,12,42,17,3,0,8,15,2,3,621,192,212,37,14,199,409,11,493,725,624,2,173,6,0,1,0,0,0,0,0,32,8,0,119,0,11,15,0,51,433,352,281,166,155,70,29,25,34,16,12,9,3,4,0,0,406,259,139,36,28,23,11,10,16,4,6,0,0,0,0,2,0,5,0,5,9,0,6,0
9,0xbbcd65,0x9138120879a122d5ce718194d3be30d0f70a2b37583f...,143881,3,360,60,83,48,8,95,62,0,130,215,310,15,7,0,22,7,35,4,12,12,42,19,3,0,8,15,2,3,648,194,229,37,15,196,404,11,505,752,617,2,167,6,0,1,0,0,0,0,0,32,8,0,145,0,11,13,0,52,450,370,275,179,155,73,30,26,32,14,11,9,4,3,0,0,425,270,141,33,26,23,11,10,16,4,6,0,0,0,0,2,0,5,0,5,9,0,6,0


In [11]:
# Show GasUsed per Transaction
df[['Block #', 'Transactions', 'Gas Used']]

,Block #,Transactions,Gas Used
0,0xbbcd65,0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90...,106777
1,0xbbcd65,0xef6102ed1c3a4841aaae5d6fbb0e5cf139584a0776be...,83381
2,0xbbcd65,0x8a7eec8c1e392d0f2d8178731061024bff3f64f4c0c9...,121450
3,0xbbcd65,0xc5ad659bbb28dc475f7cae798ae752c5cc07f5608aaf...,65866
4,0xbbcd65,0x2c015811f7241d46e583b449850af1d9faed32c036a1...,118690
5,0xbbcd65,0x20a4704294faecc2f3569764797fdc4a511cd8abef0d...,118762
6,0xbbcd65,0x32cd15e283acbb68c1f423b661feb49783c1c8914f23...,119028
7,0xbbcd65,0x911c9ece1b6418b58b9ae99a8ffef09dbf6ca018c01e...,145742
8,0xbbcd65,0xbd7c781dff21d7d5fa20734a3400fd58f05de9d8aad4...,159802
9,0xbbcd65,0x9138120879a122d5ce718194d3be30d0f70a2b37583f...,143881


In [12]:
gas_table = get_gas_table()

In [13]:
data_result = []

# Add new fields: Gas Used Rp. and Difference
for line in data:
    new_line = apply_gas_table(gas_table, line)
    if new_line:
        data_result.append(new_line)

Ignoring transaction 0xf9a29b31b92bae188dd3b4e526506d4094311a5b03ac7685497adf0b15a13289 gasUsed: 0
Ignoring transaction 0xbf52d5b98a74076431c1857dcf111ff788975bb8726b65d30fffbaeb91096c2d gasUsed: 0
Ignoring transaction 0x27741fc4ec3b62330a56e37e9a8c7a2be5af68d661827b31eab80ad5f219ac6f gasUsed: 0
Ignoring transaction 0x3b937ec01ba2810dfaf1cd05a08264d665c11b56b7a50f53c45988f9e2031185 gasUsed: 0
Ignoring transaction 0x45bbe059af21ce9c1783303e0dacfcbb9c73910a4de67f6c49cb48dda200a8cd gasUsed: 0
Ignoring transaction 0xe0142c92179aa8f30d2260442e52b8dc47b1179a218c5bd2edd1d4472b8ee19f gasUsed: 0
Ignoring transaction 0x4034edccecb561b8404c9e7d380d7a5c0e58d273adf346bbd29426b66acd1662 gasUsed: 0
Ignoring transaction 0xa1390d4c52210267934ab7cca744c5bba4c11514404e9648228e4e69c7391f2d gasUsed: 0
Ignoring transaction 0xe17093b390902be7dc6df7c70dafd6237df1003fb5c8920d78968eda9ecf4100 gasUsed: 0
Ignoring transaction 0x8092e30de606c3e0781365d58fe894c5a5243b2f554e63b4fbc2231ab02f9513 gasUsed: 0
Ignoring t

In [14]:
result_csv = convert_to_csv(data_result)
write_csv(RESULTDIR + FILE, result_csv)

In [15]:
new_columns = columns_name[0:3] + ["Gas Used RP", "Difference", "%"] + columns_name[3:]

In [16]:
df_result = pd.DataFrame(data_result[:100], columns=new_columns)

In [17]:
df_result

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%,STOP,ADD,MUL,SUB,DIV,SDIV,MOD,SMOD,ADDMOD,MULMOD,EXP,SIGNEXTEND,0x0c,0x0d,0x0e,0x0f,LT,GT,SLT,SGT,EQ,ISZERO,AND,OR,XOR,NOT,BYTE,SHL,SHR,SAR,0x1e,0x1f,KECCAK256,0x21,0x22,0x23,0x24,0x25,0x26,0x27,0x28,0x29,0x2a,0x2b,0x2c,0x2d,0x2e,0x2f,ADDRESS,BALANCE,ORIGIN,CALLER,CALLVALUE,CALLDATALOAD,CALLDATASIZE,CALLDATACOPY,CODESIZE,CODECOPY,GASPRICE,EXTCODESIZE,EXTCODECOPY,RETURNDATASIZE,RETURNDATACOPY,EXTCODEHASH,BLOCKHASH,COINBASE,TIMESTAMP,NUMBER,DIFFICULTY,GASLIMIT,CHAINID,SELFBALANCE,0x48,0x49,0x4a,0x4b,0x4c,0x4d,0x4e,0x4f,POP,MLOAD,MSTORE,MSTORE8,SLOAD,SSTORE,JUMP,JUMPI,PC,MSIZE,GAS,JUMPDEST,0x5c,0x5d,0x5e,0x5f,PUSH1,PUSH2,PUSH3,PUSH4,PUSH5,PUSH6,PUSH7,PUSH8,PUSH9,PUSH10,PUSH11,PUSH12,PUSH13,PUSH14,PUSH15,PUSH16,PUSH17,PUSH18,PUSH19,PUSH20,PUSH21,PUSH22,PUSH23,PUSH24,PUSH25,PUSH26,PUSH27,PUSH28,PUSH29,PUSH30,PUSH31,PUSH32,DUP1,DUP2,DUP3,DUP4,DUP5,DUP6,DUP7,DUP8,DUP9,DUP10,DUP11,DUP12,DUP13,DUP14,DUP15,DUP16,SWAP1,SWAP2,SWAP3,SWAP4,SWAP5,SWAP6,SWAP7,SWAP8,SWAP9,SWAP10,SWAP11,SWAP12,SWAP13,SWAP14,SWAP15,SWAP16,LOG0,LOG1,LOG2,LOG3,LOG4,0xa5,0xa6,0xa7,0xa8,0xa9,0xaa,0xab,0xac,0xad,0xae,0xaf,0xb0,0xb1,0xb2,0xb3,0xb4,0xb5,0xb6,0xb7,0xb8,0xb9,0xba,0xbb,0xbc,0xbd,0xbe,0xbf,0xc0,0xc1,0xc2,0xc3,0xc4,0xc5,0xc6,0xc7,0xc8,0xc9,0xca,0xcb,0xcc,0xcd,0xce,0xcf,0xd0,0xd1,0xd2,0xd3,0xd4,0xd5,0xd6,0xd7,0xd8,0xd9,0xda,0xdb,0xdc,0xdd,0xde,0xdf,0xe0,0xe1,0xe2,0xe3,0xe4,0xe5,0xe6,0xe7,0xe8,0xe9,0xea,0xeb,0xec,0xed,0xee,0xef,CREATE,CALL,CALLCODE,RETURN,DELEGATECALL,CREATE2,0xf6,0xf7,0xf8,0xf9,STATICCALL,0xfb,0xfc,REVERT,INVALID,SELFDESTRUCT
0,0xbbcd65,0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90...,106777,100023,6754,6.33%,2,187,39,47,22,0,0,0,0,0,1,0,0,0,0,0,56,36,2,0,61,117,177,9,0,3,0,13,5,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,6,9,27,13,3,0,3,0,6,0,8,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,351,114,126,0,21,10,122,218,0,0,7,288,0,0,0,0,414,348,1,85,4,0,0,1,0,0,0,8,1,19,5,0,0,0,0,76,0,0,0,0,0,0,0,8,7,0,0,30,239,212,150,102,88,41,17,15,18,9,6,3,1,2,0,0,231,139,85,27,18,14,6,5,8,2,3,0,0,0,0,0,0,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,6,0,0,0,0,0,0,3,0,0,0,0,0
1,0xbbcd65,0xef6102ed1c3a4841aaae5d6fbb0e5cf139584a0776be...,83381,76505,6876,8.25%,1,210,38,47,26,0,0,0,0,0,5,0,0,0,0,0,54,32,0,0,75,127,175,9,0,5,0,11,3,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,7,7,28,10,3,0,0,0,4,0,11,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,352,115,128,0,20,9,110,224,0,0,6,267,0,0,0,0,424,343,1,94,4,0,0,1,0,0,0,0,0,19,5,0,0,0,0,80,0,0,0,0,0,0,0,9,8,0,0,31,254,208,157,95,83,37,22,12,17,9,7,5,1,2,0,0,236,156,82,19,14,13,6,4,8,2,3,0,0,0,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,6,0,0,0,0,0,0,3,0,0,0,0,0
2,0xbbcd65,0x8a7eec8c1e392d0f2d8178731061024bff3f64f4c0c9...,121450,114766,6684,5.50%,2,195,39,45,25,0,0,0,0,0,2,0,0,0,0,0,55,31,0,0,82,127,172,7,0,3,0,13,3,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,9,9,27,11,3,0,0,0,6,0,8,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,345,111,120,0,19,10,104,229,0,0,7,261,0,0,0,0,415,339,1,106,4,0,0,1,0,0,0,0,0,19,5,0,0,0,0,83,0,0,0,0,0,0,0,8,9,0,0,28,250,208,149,89,81,35,17,13,18,9,6,3,1,2,0,0,218,135,77,17,13,12,5,5,8,2,3,0,0,0,0,0,0,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,6,0,0,0,0,0,0,3,0,0,0,0,0
3,0xbbcd65,0xc5ad659bbb28dc475f7cae798ae752c5cc07f5608aaf...,65866,60376,5490,8.34%,2,158,28,47,17,0,0,0,0,0,20,0,0,0,0,0,33,15,0,0,86,93,110,3,0,4,1,7,2,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,8,7,44,12,6,0,1,0,5,0,5,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,280,100,123,0,20,0,82,187,0,0,8,220,0,0,0,0,419,280,0,97,2,1,0,1,0,0,0,0,1,3,1,0,0,0,0,45,0,0,0,0,0,0,0,1,7,0,1,20,203,184,81,86,56,25,26,24,18,8,4,2,0,0,0,0,189,109,47,14,19,17,11

In [18]:
df_result.loc[:, (df_result != 0).any(axis=0)]

,Block #,Transactions,Gas Used,Gas Used RP,Difference,%,STOP,ADD,MUL,SUB,DIV,EXP,LT,GT,SLT,EQ,ISZERO,AND,OR,NOT,BYTE,SHL,SHR,KECCAK256,ADDRESS,CALLER,CALLVALUE,CALLDATALOAD,CALLDATASIZE,CALLDATACOPY,CODECOPY,EXTCODESIZE,RETURNDATASIZE,RETURNDATACOPY,TIMESTAMP,POP,MLOAD,MSTORE,SLOAD,SSTORE,JUMP,JUMPI,GAS,JUMPDEST,PUSH1,PUSH2,PUSH3,PUSH4,PUSH5,PUSH6,PUSH8,PUSH9,PUSH10,PUSH11,PUSH12,PUSH13,PUSH14,PUSH15,PUSH18,PUSH20,PUSH21,PUSH28,PUSH29,PUSH31,PUSH32,DUP1,DUP2,DUP3,DUP4,DUP5,DUP6,DUP7,DUP8,DUP9,DUP10,DUP11,DUP12,DUP13,DUP14,DUP15,DUP16,SWAP1,SWAP2,SWAP3,SWAP4,SWAP5,SWAP6,SWAP7,SWAP8,SWAP9,SWAP10,SWAP11,SWAP12,SWAP13,SWAP14,SWAP15,LOG1,LOG2,LOG3,LOG4,CALL,RETURN,DELEGATECALL,STATICCALL,REVERT
0,0xbbcd65,0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90...,106777,100023,6754,6.33%,2,187,39,47,22,1,56,36,2,61,117,177,9,3,0,13,5,18,2,6,9,27,13,3,3,6,8,1,2,351,114,126,21,10,122,218,7,288,414,348,1,85,4,0,1,0,0,0,8,1,19,5,0,76,0,8,7,0,30,239,212,150,102,88,41,17,15,18,9,6,3,1,2,0,0,231,139,85,27,18,14,6,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
1,0xbbcd65,0xef6102ed1c3a4841aaae5d6fbb0e5cf139584a0776be...,83381,76505,6876,8.25%,1,210,38,47,26,5,54,32,0,75,127,175,9,5,0,11,3,18,2,7,7,28,10,3,0,4,11,2,2,352,115,128,20,9,110,224,6,267,424,343,1,94,4,0,1,0,0,0,0,0,19,5,0,80,0,9,8,0,31,254,208,157,95,83,37,22,12,17,9,7,5,1,2,0,0,236,156,82,19,14,13,6,4,8,2,3,0,0,0,0,1,0,3,0,3,6,0,3,0
2,0xbbcd65,0x8a7eec8c1e392d0f2d8178731061024bff3f64f4c0c9...,121450,114766,6684,5.50%,2,195,39,45,25,2,55,31,0,82,127,172,7,3,0,13,3,17,2,9,9,27,11,3,0,6,8,1,2,345,111,120,19,10,104,229,7,261,415,339,1,106,4,0,1,0,0,0,0,0,19,5,0,83,0,8,9,0,28,250,208,149,89,81,35,17,13,18,9,6,3,1,2,0,0,218,135,77,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
3,0xbbcd65,0xc5ad659bbb28dc475f7cae798ae752c5cc07f5608aaf...,65866,60376,5490,8.34%,2,158,28,47,17,20,33,15,0,86,93,110,3,4,1,7,2,16,1,8,7,44,12,6,1,5,5,1,1,280,100,123,20,0,82,187,8,220,419,280,0,97,2,1,1,0,0,0,0,1,3,1,0,45,0,1,7,1,20,203,184,81,86,56,25,26,24,18,8,4,2,0,0,0,0,189,109,47,14,19,17,11,4,5,1,2,0,0,0,0,1,0,0,0,6,6,1,1,1
4,0xbbcd65,0x2c015811f7241d46e583b449850af1d9faed32c036a1...,118690,112303,6387,5.38%,2,183,39,52,22,1,57,33,0,61,115,154,7,3,0,18,5,17,2,9,9,27,13,3,0,6,8,1,2,334,112,122,18,10,104,211,7,261,412,320,1,84,4,0,1,0,0,0,0,0,19,5,0,65,0,8,7,0,29,228,214,146,88,82,36,16,13,18,9,6,3,1,2,0,0,212,124,81,21,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
5,0xbbcd65,0x20a4704294faecc2f3569764797fdc4a511cd8abef0d...,118762,112224,6538,5.51%,2,195,39,47,22,1,57,32,0,64,118,169,7,3,0,13,5,16,2,6,9,27,13,3,0,6,8,1,2,351,111,118,17,10,104,213,7,262,408,322,1,87,4,0,1,0,0,0,0,0,19,5,0,83,0,8,7,0,28,242,212,147,88,84,36,17,13,18,9,6,3,1,2,0,0,213,135,79,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
6,0xbbcd65,0x32cd15e283acbb68c1f423b661feb49783c1c8914f23...,119028,112382,6646,5.58%,2,195,39,47,24,1,57,30,0,76,126,171,7,3,0,13,3,16,2,6,9,27,13,3,0,6,8,1,2,351,111,118,17,10,104,223,7,262,406,332,1,99,4,0,1,0,0,0,0,0,19,5,0,83,0,8,9,0,28,252,212,147,88,84,36,17,13,18,9,6,3,1,2,0,0,215,135,79,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
7,0xbbcd65,0x911c9ece1b6418b58b9ae99a8ffef09dbf6ca018c01e...,145742,136035,9707,6.66%,4,231,30,59,44,12,60,53,0,82,159,234,10,5,0,9,6,28,4,11,10,35,17,1,1,7,14,2,2,665,122,189,44,13,195,282,10,436,690,495,1,117,4,0,0,0,0,6,0,0,19,5,0,118,0,9,8,0,46,360,277,190,168,86,56,24,11,16,10,16,6,5,1,0,0,388,230,118,44,19,18,4,4,4,1,8,1,0,0,0,1,1,4,0,6,7,0,5,0
8,0xbbcd65,0xbd7c781dff21d7d5fa20734a3400fd58f05de9d8aad4...,159802,147932,11870,7.43%,3,331,61,87,52,15,94,56,0,138,217,294,15,7,0,22,5,30,5,9,12,42,17,3,0,8,15,2,3,621,192,212,37,14,199,409,11,493,725,624,2,173,6,0,1,0,0,0,0,0,32,8,0,119,0,11,15,0,51,433,352,281,166,155,70,29,25,34,16,12,9,3,4,0,0,406,259,139,36,28,23,11,10,16,4,6,0,0,0,0,2,0,5,0,5,9,0,6,0
9,0xbbcd65,0x9138120879a122d5ce718194d3be30d0f70a2b37583f...,143881,131695,12186,8.47%,3,360,60,83,48,8,95,62,0,130,215,310,15,7,0,22,7,35,4,12,12,42,19,3,0,8,15,2,3,648,194,229,37,15,196,404,11,505,

#### 